In [69]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [58]:
ratings = pd.read_csv('BX-CSV-Dump/BX-Book-Ratings.csv', encoding='windows-1251', sep=';')
books = pd.read_csv('BX-CSV-Dump/BX-Books.csv', encoding='windows-1251', sep=';',
                   usecols=['ISBN',
                            'Book-Title',
                            'Book-Author',
                            'Year-Of-Publication',
                            'Publisher',
                            'Image-URL-S',
                            'Image-URL-M',
                            'Image-URL-L'])
users = pd.read_csv('BX-CSV-Dump/BX-Users.csv', encoding='windows-1251', sep=';')

ratings = ratings.rename(columns={'Book-Rating': 'Rating'})
ratings["Rating"] = ratings["Rating"].astype("int8")
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
books = books.rename(columns={'Book-Title': 'Title', 'Book-Author': 'Author'})

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Средние значения и количество

In [59]:
avg_user_rating = ratings.groupby('User-ID')['Rating'].mean()
cnt_user_rating = ratings.groupby('User-ID')['Rating'].count()

avg_book_rating = ratings.groupby('ISBN')['Rating'].mean()
cnt_book_rating = ratings.groupby('ISBN')['Rating'].count()

avg_user_rating.name = 'avg_rating'
cnt_user_rating.name = 'N_ratings'
avg_book_rating.name = 'avg_rating'
cnt_book_rating.name = 'N_ratings'

In [60]:
users = users.merge(avg_user_rating, on=['User-ID'])
users = users.merge(cnt_user_rating, on=['User-ID'])

books = books.merge(avg_book_rating, on=['ISBN'])
books = books.merge(cnt_book_rating, on=['ISBN'])

# KNN model - recommend similar book
## Find the most similar book (using user-recommendation vectors) and recommend it.



In [67]:
pd_matrix = \
    pd.merge(books.loc[books["N_ratings"] > 20, "ISBN"], 
             ratings, how="left", left_on="ISBN", right_on="ISBN").drop_duplicates()

pd_matrix = pd_matrix.pivot(index='ISBN', columns='User-ID', values='Rating').fillna(0).astype('int8')
pd_matrix

User-ID,8,9,10,14,16,17,19,26,32,39,...,278831,278832,278836,278838,278843,278844,278846,278849,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000649840X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006547834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006550576,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006550789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0007110928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8845205118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8845247414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
884590184X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# Change to sparse matrix if we didn't have enough memory
matrix = csr_matrix(pd_matrix.values)

In [71]:
matrix

<6863x55167 sparse matrix of type '<class 'numpy.int8'>'
	with 147020 stored elements in Compressed Sparse Row format>

In [73]:
# Create a model
N_predicted_neighbours = 11
KNN = NearestNeighbors(metric='cosine', n_neighbors=N_predicted_neighbours, n_jobs=-1)
# Fit the model
KNN.fit(matrix)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=11)

In [80]:
# Predict
distances, indices = KNN.kneighbors(matrix)
distances.shape, indices.shape

((6863, 11), (6863, 11))

In [84]:
# Predictions

print(f"Because you liked {books.loc[books['ISBN'] == pd_matrix.index[indices[4865][0]], 'Title'].values[0]} you may like:")
print()
for i in range(1, N_predicted_neighbours):
    print(f"{books.loc[books['ISBN'] == pd_matrix.index[indices[4865][i]], 'Title'].values[0]} with distance {distances[4865][i]:.3f}.")

Because you liked Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)) you may like:

Harry Potter and the Chamber of Secrets (Book 2) with distance 0.701.
Harry Potter and the Prisoner of Azkaban (Book 3) with distance 0.760.
Harry Potter and the Goblet of Fire (Book 4) with distance 0.828.
Harry Potter and the Goblet of Fire (Book 4) with distance 0.838.
Harry Potter and the Order of the Phoenix (Book 5) with distance 0.852.
Harry Potter and the Prisoner of Azkaban (Book 3) with distance 0.869.
Harry Potter and the Chamber of Secrets (Book 2) with distance 0.875.
The Lost King (Star of the Guardians, Vol 1) with distance 0.902.
Fried Green Tomatoes at the Whistle Stop Cafe with distance 0.904.
Love Story with distance 0.911.


In [85]:
def recommend_similar_book(isbn, indices, ratings_matrix, books_table, N_recommendations=1, distances=None):
    """
    Recommends a book title.
    
    Parameters
    ----------
    ISBN: str
        ISBN of a book a user liked
    indices: np.array
        indices of ratings_matrix as predicted by KNN
    ratings_matrix: pd.DataFrame
        user-book-rating matrix with ratings as values
    N_recommendations: int (default 1)
        How many books to recommend?
    distances: np.array
        How distant are books from each other by KNN?
    """
    # TODO: This should be rather split in separate variables, this reads terribly
    print(f"Because you liked {books_table.loc[books_table['ISBN'] == ratings_matrix.index[indices[isbn][0]], 'Title'].values[0]} you may like:")
    print()
    for i in range(1, 1+N_recommendations):
        if distances:
            print(f"{books_table.loc[books_table['ISBN'] == ratings_matrix.index[indices[isbn][i]], 'Title'].values[0]} with distance {distances[isbn][i]:.3f}.")
        else:
            print(f"{books_table.loc[books_table['ISBN'] == ratings_matrix.index[indices[isbn][i]], 'Title'].values[0]}.")

In [86]:
recommend_similar_book(4865, indices, pd_matrix, books)

Because you liked Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)) you may like:

Harry Potter and the Chamber of Secrets (Book 2).


# KNN model - recommend similar user's favourite book
## Find the most similar user and recommend his top rated books that the current user have not read yet.

In [87]:
# Create a model
KNN2 = NearestNeighbors(metric='cosine', n_neighbors=20, n_jobs=-1)

In [88]:
# Fit
KNN2.fit(matrix.T)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=20)

In [92]:
# Predict
distances2, indices2 = KNN2.kneighbors(matrix.T)

In [93]:
def recommend_favourite_book_of_similar_user(userID, indices, ratings_matrix, users_table, books_table, ratings_table, N_recommendations=1, distances=None):
    """
    Recommends a book title based on favourite books of ten most similar users.
    
    The order of books is following:
    Take the most similar user, sort his books by rating,
    exclude everything the current predicted user already read.
    Output books one by one.
    If there is only a few books from the most similar user and
    we run out of books, take next similar user and output
    his favorite books in a similar fashion.
    
    Parameters
    ----------
    userID: int
        ID of a user we want a recommendation for
    indices: np.array
        indices of ratings_matrix as predicted by KNN
    ratings_matrix: pd.DataFrame
        user-book-rating matrix with ratings as values
    users_table: pd.DataFrame
        Information about users
    books_table: pd.DataFrame
        Information about books
    ratings_table: pd.DataFrame
        Information about ratings
    N_recommendations: int (default 1)
        How many books to recommend?
    distances: np.array
        How distant are books from each other by KNN?
    """
    selected_index = ratings_matrix.columns.get_loc(userID)
    already_read_book_isbns = list(ratings_table.loc[ratings_table["User-ID"] == userID, "ISBN"].values)
    not_read_books = ratings_table.loc[~ratings_table["ISBN"].isin(already_read_book_isbns)]
    books_to_recommend = list()
    for i in range(1,10):
        similar_user_index = indices[selected_index][i]
        similar_user_ID = ratings_matrix.columns[similar_user_index]
        possible_to_recommend = not_read_books.loc[not_read_books["User-ID"] == similar_user_ID]
        possible_to_recommend = possible_to_recommend.sort_values(by="Rating", ascending=False)
        for a, row in possible_to_recommend.iterrows():
            books_to_recommend.append(books_table.loc[books["ISBN"] == row["ISBN"], "Title"].values[0])
            if len(books_to_recommend) > N_recommendations-1:
                break
        if len(books_to_recommend) > N_recommendations-1:
            break
    print(f"Based on users who like similar books as you, you may like:")
    print()
    for book_name in books_to_recommend:
        print(book_name)

In [94]:
recommend_favourite_book_of_similar_user(175003, 
                                         indices2, 
                                         pd_matrix, 
                                         users, 
                                         books, 
                                         ratings, 
                                         N_recommendations=3, 
                                         distances=distances2)

Based on users who like similar books as you, you may like:

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Prisoner of Azkaban (Book 3)
Ultimate Unofficial Guide to the Mysteries of Harry Potter
